<a href="https://colab.research.google.com/github/RuinedPenguin/SMAI-IIITH-Project-Lenet/blob/object-oriented-approach/src/Lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cupy as cp

In [2]:
from abc import ABC, abstractmethod

In [21]:
# pip install zope
import zope.interface

## **Lenet**

In [3]:
class Layer(ABC):

  @abstractmethod
  def initialise_weights(self, input_shape):
    pass

  @abstractmethod
  def forward(self, x):
    pass

  @abstractmethod
  def backward(self):
    pass

In [64]:
class Conv(Layer):
  def __init__(self, k, c_out):
    self.k = k
    self.kernels = []
    # self.p = p
    # self.s = s

  def initialise_weights(self, input_shape):
    for i in range(self.c_out):
      self.kernels.append(cp.random.rand(self.k))
    return (input_shape[0] - self.k[0] + 1, 
            input_shape[1] - self.k[1] + 1, 
            self.c_out)

  def forward(self, x):
    new_h = x.shape[0] - self.k[0] + 1
    new_w = x.shape[1] - self.k[1] + 1

    # if self.c_out == 1:
    #   return convolve(x, self.kernels[0], new_h, new_w)
    
    out = cp.zeros((new_h, new_w, self.c_out))

    for i in range(self.c_out):
      out[:,:,i] = convolve(x, self.kernels[i], new_h, new_w)

    return out

  def convolve(inp, kernel, new_h, new_w):
    out = cp.zeros((new_h, new_w))

    for i in range(new_h):
      for j in range(new_w):
        out[i,j] = inp[i : i +kernel.shape[0], j : j +kernel.shape[0]] * kernel

    return out

  def backward(self):
    pass

In [66]:
class Pool(Layer):
  def __init__(self, k):
    self.k = k
    # self.p = p
    # self.s = s

  def initialise_weights(self, input_shape):
    if len(input_shape) == 2:
      return (input_shape[0] // self.k, input_shape[1] // self.k)
    
    return (input_shape[0] // self.k, input_shape[1] // self.k, 
            input_shape[2])
    

  def forward(self, x):
    if len(x.shape) == 2:
      return self.pool_2d(x)
    
    out = cp.zeros((x.shape[0] // self.k, x.shape[1] // self.k, x.shape[2]))
    for i in range(x.shape[2]):
      out[:,:,i] = self.pool_2d(x[:,:,i])
    
    return out

  def pool_2d(self, x_2d):
    out = cp.zeros((x_2d.shape[0] // self.k, x_2d.shape[1] // self.k))
    for i in range(out.shape[0]):
      for j in range(out.shape[1]):
        h = i * self.k
        w = j * self.k
        out[i,j] = self.pooling_function(x_2d[h:h+self.k, w:w+self.k])

    return out

  @abstractmethod
  def pooling_function(self, matrix):
    pass

  def backward(self):
    pass

In [48]:
class MaxPool(Pool):
  def __init__(self, k):
    super().__init__(k)

  def pooling_function(self, matrix):
    return cp.max(matrix)

In [60]:
class Flatten(Layer):
  def initialise_weights(self, input_shape):
    out_shape = 1
    for i in input_shape:
    out_shape *= i
    return out_shape    # returns int

  def forward(self, x):
    return x.reshape((-1,1))

  def backward(self):
    pass

In [70]:
class Fully_Connected(Layer):
  def __init__(self, units):
    self.units = units
    self.weights

  def initialise_weights(self, input_shape):
    self.weights = cp.random.rand((input_shape, self.units))
    return self.units    # returns int

  def forward(self, x):
    return cp.dot(weights, x)

  def backward(self):
    pass

In [51]:
a = MaxPool(3)

In [52]:
a.forward(cp.array(cp.zeros((4,4,2))))

array([[[0., 0.]]])

In [61]:
f = Flatten()

In [62]:
f.forward(cp.array(cp.zeros((4,4,2))))

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [71]:
class Model:
  def __init__():
    self.layers = []

  def addLayer(self, layer):
    self.layers.append(layer)

  def compile(self, x_shape, y_shape):
    input_shape = x_shape
    for layer in self.layers:
      input_shape = layer.initialise_weights(x_shape)

    if input_shape == y_shape:
      print("The layers fit correctly")
    else:
      print("The layers don't fit correctly")
     

  def fit(self, x_train, y_train, epochs, validation_data):
    for epoch in range(epochs):
      layer_in = x_train

      for layer in self.layers:
        layer_in = layer.forward(layer_in)
    return layer_in

In [69]:
cp.array([[1,2], [3,4]]) * cp.array([[[1,2], [3,4]], [[1,3], [3,4]]])

array([[[ 1,  4],
        [ 9, 16]],

       [[ 1,  6],
        [ 9, 16]]])